In [1]:
import warnings
warnings.filterwarnings('ignore')
import folium
import json
import glob
import os
import pandas as pd
import numpy as np

In [ ]:
path = 'Datas/'
allFiles = glob.glob(os.path.join(path+"*.csv"))

df = pd.DataFrame()
for file in allFiles:

    print(file)

    temp = pd.read_csv(file, engine='python', encoding='utf-8')
    df = pd.concat([df, temp])

Datas\소상공인시장진흥공단_상가(상권)정보_경기_202109.csv


In [ ]:
print('컬럼 항목 수 :', len(df.columns))
list(df.columns)

In [ ]:
print('='*70)
print('상권업종대분류명', set(df['상권업종대분류명']))
print('='*70)
print('상권업종중분류명', set(df['상권업종중분류명']))
print('='*70)

In [ ]:
dataset = df[['상호명','지점명',
              '상권업종대분류명', '상권업종중분류명',
              '시도명', '시군구명', '행정동명',
              '위도', '경도']]

dataset.head()

In [ ]:
df_fastfood = dataset[(dataset['상권업종중분류명']=='패스트푸드')&(dataset['시도명']=='서울특별시')]
df_fastfood.index = range(len(df_fastfood))
print('서울시 내 패스트푸드 점포 수 :', len(df_fastfood))
df_fastfood.head()

In [ ]:
df_seoul_mac = df_fastfood[df_fastfood['상호명'].str.contains('맥도날드')]
df_seoul_mac.index = range(len(df_seoul_mac))
print('서울시 내 맥도날드 점포 수 :', len(df_seoul_mac))
df_seoul_mac.head()

In [ ]:
df_seoul_bk = df_fastfood[df_fastfood['상호명'].str.contains('버거킹')]
df_seoul_bk.index = range(len(df_seoul_bk))
print('서울시 내 버거킹 점포 수 :', len(df_seoul_bk))
df_seoul_bk.head()

In [ ]:
mac_gu = df_seoul_mac.groupby('시군구명')['상호명'].count().to_frame().sort_values(by='상호명', ascending=False)
mac_gu = mac_gu.reset_index()
mac_gu = mac_gu.set_index('시군구명')
mac_gu

In [ ]:
bk_gu = df_seoul_bk.groupby('시군구명')['상호명'].count().to_frame().sort_values(by='상호명', ascending=False) #내림차순
bk_gu = bk_gu.reset_index()
bk_gu = bk_gu.set_index('시군구명')
bk_gu

In [ ]:
geo_path = 'Datas/02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [ ]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)
data_size = len(df_seoul_mac)

# 지도 정의
map_mac = folium.Map(location=loc, zoom_start=12)
map_mac.choropleth('geo_data'=geo_str,
              data = mac_gu['상호명'],
              columns=[mac_gu.index, mac_gu['상호명']],
              fill_color='YlGnBu',
              key_on='feature.id')


# 포인트 마커 추가

for i in range(data_size):

    folium.Marker(list(df_seoul_mac.iloc[i][['위도', '경도']]),
                 popup=df_seoul_mac.iloc[i][['지점명']],
                 icon=folium.Icon(color='green')).add_to(map_mac)

map_mac

In [ ]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)
data2_size = len(df_seoul_bk)

# 지도 정의
map_bk = folium.Map(location=loc, zoom_start=12)
map_bk.choropleth(geo_data=geo_str,
              data = bk_gu['상호명'],
              columns=[bk_gu.index, bk_gu['상호명']],
              fill_color='YlGnBu', #색상정보(바랭이)
              key_on='feature.id') #지리정보와 시각화정보의 공통변수로서 'feature.id'


# 포인트 마커 추가
for i in range(data2_size):

    folium.Marker(list(df_seoul_bk.iloc[i][['위도', '경도']]),
                 popup=df_seoul_bk.iloc[i][['지점명']],
                 icon=folium.Icon(color='red')).add_to(map_bk)

map_bk

In [ ]:
# 위치 파라미터 설정
loc = [37.5502, 126.982] # 위도(N), 경도(E)(서울)

data_size = len(df_seoul_mac)
data2_size = len(df_seoul_bk)

# 지도 정의
map = folium.Map(location=loc,
                 tiles = 'Stamen Toner', #하천과 길로 표시된 지도
                 zoom_start=12)

# 포인트 마커 추가

for i in range(data_size):

    folium.Marker(list(df_seoul_mac.iloc[i][['위도', '경도']]),
                 popup=df_seoul_mac.iloc[i][['지점명']],
                 icon=folium.Icon(color='green')).add_to(map)


for i in range(data2_size):

    folium.Marker(list(df_seoul_bk.iloc[i][['위도', '경도']]),
                 popup=df_seoul_bk.iloc[i][['지점명']],
                 icon=folium.Icon(color='red')).add_to(map)


map

In [ ]:
# 위치 파라미터 설정
# 강남역 좌표
loc = [37.497895, 127.027565] # 위도(N), 경도(E)
data_size = len(df_seoul_mac)
data2_size = len(df_seoul_bk)

# 지도 정의
map = folium.Map(location=loc,
                 tiles = 'Stamen Toner', # 'OpenStreetMap'
                 zoom_start=12)

# 포인트 서클 추가

for i in range(data_size):

    folium.Circle(list(df_seoul_mac.iloc[i][['위도', '경도']]),
                  radius = 100,
                  popup = df_seoul_mac.iloc[i]['지점명'],
                  color = '#2c9147',fill_color = '#2c9147').add_to(map)

for i in range(data2_size):

    folium.Circle(list(df_seoul_bk.iloc[i][['위도', '경도']]),
                  radius = 100,
                  popup = df_seoul_bk.iloc[i]['지점명'],
                  color = '#91332c',fill_color = '#91332c').add_to(map)

map